<a href="https://colab.research.google.com/github/junieberry/NLP-withPyTorch/blob/main/03_Yelp/03_Yelp_Classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

공부한 코드 : https://github.com/rickiepark/nlp-with-pytorch/tree/main/chapter_3


함께한 재생목록 : https://www.youtube.com/watch?v=L3Chu2A5TyU

In [19]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [20]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

## 3.6 레스토랑 리뷰 감성 분류하기

### 3.6.2  파이토치 데이터셋 이해하기

파이토치는 Dataset 클래스로 데이터셋을 추상화한다.

Dataset은 `__getitem()`과 `__len()` 메서드를 구현한다.

In [21]:
class ReviewDataset(Dataset):
  ## review_df (pandas.DataFrame) == 데이터셋
  ## vectorizer (ReviewVectorizer) == ReviewVectorizer 객체
  def __init__(self, review_df, vectorizer):
    self.review_df = review_df
    self._vectorizer = vectorizer

    self.train_df = self.review_df[self.review_df.split=='train']
    self.train_size = len(self.train_df)

    self.val_df = self.review_df[self.review_df.split=='val']
    self.validation_size = len(self.val_df)

    self.test_df = self.review_df[self.review_df.split=='test']
    self.test_size = len(self.test_df)

    self._lookup_dict = {'train': (self.train_df, self.train_size),
                          'val': (self.val_df, self.validation_size),
                         'test': (self.test_df, self.test_size)}

    self.set_split('train')
  

  ## classmethod는 static과 같다고 볼 수 있따..
  @classmethod

  ## 데이터셋 로드하고 새 RevewVectorizer을 만든당
  ## review_csv (str) == 데이터셋 위치
  ## return == ReviewDataset의 인스턴스
  def load_dataset_and_make_vectorizer(cls, review_csv):
    review_df = pd.read_csv(review_csv)
    train_review_df = review_df[review_df.split=='train']
    return cls(review_df,ReviewVectorizer.from_dataframe(train_review_df))

  @classmethod
  ## 데이터셋 로드하고 ReviewVectorizer 재사용
  ## review_csv (str) == 데이터셋 위치
  ## vectorizer_filepath (str) == ReviewVectorizer 객체의 저장 위치
  ## return == ReviewDataset의 인스턴스

  def load_Dataset_and_load_vectorizer(cls, review_csv, vectorizer_filepath):
    review_df = pd.read_csv(review_csv)
    vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
    return cls(review_df, vectorizer)
  
  @staticmethod
  ## 파일에서 ReviewVectorizer 로드
  ## vectorizer_filepath (str) == 직렬화된 ReviewVectorizer 객체의 위치
  ## return == ReviewVectorizer의 인스턴스
  def load_vectorizer_only(vectorizer_filepath):
    with open(vectorizer_filepath) as fp:
        return ReviewVectorizer.from_serializable(json.load(fp))
  
  ## ReviewVectorizer 객체를 json 형태로 저장
  ## vectorizer_filepath (str) == ReviewVectorizer의 저장 위치
  def save_vectorizer(self, vectorizer_filepath):
    with open(vectorizer_filepath, "w") as fp:
      json.dump(self._vectorizer.to_serializable(), fp)
  
  ## 백터 변환 객체 반환
  def get_vectorizer(self):
    return self._vectorizer
  

  ## 데이터 프레임에 있는 열을 사용해 분할 세트 선택
  ## split (str) == "train", "val", "test" 중 하나
  def set_split(self, split="train"):
    self._target_split = split
    self._target_df, self._target_size = self._lookup_dict[split]

  def __len__(self):
    return self._target_size

  
  ## 데이터셋의 진입 메서드
  ## index (int) == 데이터 포인트 인덱스
  ## return == x_data와 y_target으로 이루어진 딕셔너리
  def __getitem__(self, index):
    row = self._target_df.iloc[index]

    review_vector = self._vectorizer.vectorize(row.review)
    rating_index = self._vectorizer.rating_vocab.lookup_token(row.rating)

    return {'x_data' : review_vector, 'y_target':rating_index}
  
  ## 배치 크기를 받아 배치 개수를 반환
  def get_num_batches(self, batch_size):
    return len(self)//batch_size


### 3.6.3 Vocabulary, Vectorizer, Dataloader

토큰을 정수에 매핑하고, 이 매핑을 각 데이터 포인트에 적용해 백터 형태로 변환

그 후 벡터로 변환한 데이터 포인트를 모델을 위해 미니배치로 모음

Vocabulary = 정수-토큰 매핑

Vocabulary = 텍스트 토큰과 클래스 레이블 정수로 매핑

Vectorizer = 어휘 사전 캡츌화, 문자열 데이터를 수치 벡터로 변환

DataLoader = 개별 벡터 데이터 포인트를 미니배치로

**Vocabulary**

토큰을 정수로 매핑!

- 토큰이 추가되면 자동으로 인덱스 증가
- UNK 토큰 사용


- `add_token` = vocabulary에 새로운 토큰 추가
- `lookup_token` = 토큰에 해당하는 인덱스 추가
- `lookup_index` = 특정 인덱스에 해당하는 토큰 추가



In [22]:
class Vocabulary(object):

  ##token_to_idx (dict): 기존 토큰-인덱스 매핑 딕셔너리
  ## add_unk (bool): UNK 토큰을 추가할지 지정하는 플래그
  ## unk_token (str): Vocabulary에 추가할 UNK 토큰
  def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
    if token_to_idx is None:
      token_to_idx = {}
    
    self._token_to_idx = token_to_idx

    self._idx_to_token = {idx: token 
                          for token, idx in self._token_to_idx.items()}
    
    self._add_unk = add_unk
    self._unk_token = unk_token

    self.unk_index = -1
    if add_unk:
      self.unk_index = self.add_token(unk_token)
  
  ## 직렬화할 수 있는 딕셔너리를 반환
  def to_serializable(self):
    return { 'token_to_idx': self._token_to_idx,
            'add_unk': self._add_unk,
            'unk_token': self._unk_token
    }
  
  @classmethod
  ## 직렬화된 딕셔너리에서 Vocabulary 객체를 만든다.
  def from_serializable(cls, contents):
    return cls(**contents)
  
  ## 토큰 추가하고 매핑 딕셔너리를 업데이트
  ## token (str) == Vocabulary에 추가할 토큰
  ## return == 토큰의 index
  def add_token(self, token):
    if token in self._token_to_idx:
      index = self._token_to_idx[token]
    else:
      index = len(self._token_to_idx)
      self._token_to_idx[token] = index
      self._idx_to_token[index] = token
    return index


  ## 토큰들을 추가하고 매핑 딕셔너리를 업데이트
  ## tokens (list) == 문자열 토큰 리스트
  ## return == 토큰들의 indices
  def add_many(self, tokens):
    return [self.add_token(token) for token in tokens]


  ## 토큰에 대응하는 인덱스 추출, 토큰이 없으면 UNK 인덱스 반환
  ## token (str) == 찾을 토큰
  ## index (int) == 토큰에 해당하는 인덱스
  def lookup_token(self, token):
        
        ## ??????
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

  ## 인덱스에 해당하는 토큰 반환
  ## index (int) == 찾을 인덱스
  ## return = 인덱스의 토큰
  def lookup_index(self, index):
    if index not in self._idx_to_token:
      raise KeyError("Vocabulary에 인덱스(%d)가 없습니다." % index)
    return self._idx_to_token[index]

  def __str__(self):
    return "<Vocabulary(size=%d)>" % len(self)

  def __len__(self):
    return len(self._token_to_idx)



**Vectorizer**

입력 데이터의 토큰을 순회하며 각 토큰을 정수로 바꿔 벡터로 만든다.

이때 벡터들의 크기는 모두 같아야한다!


 `from_dataframe()` = Dataframe을 순회하며 Vectorizer 클래스 초기화
  1. 데이터셋에 있는 모든 토큰의 빈도수 카운트
  2. cutoff보다 빈도수가 높은 Vocabulary 객체만 남김

`vectorize()` = Vectorizer 클래스의 핵심 기능 캡슐화

  - 매개변수로 리뷰 문자열을 받고 리뷰의 벡터 표현을 반환 (원핫 벡터)

  - 이때 벡터 표현은 희소 표현이며 BoW 방식이다.

In [23]:
class ReviewVectorizer(object):

  ## review_vocab (Vocabulary) == 단어를 정수에 매핑하는 Vocabulary
  ## rating_vocab (Vocabulary) == 클래스 레이블을 정수에 매핑하는 Vocabulary
  def __init__(self, review_vocab, rating_vocab):
    self.review_vocab = review_vocab
    self.rating_vocab = rating_vocab
  

  ## 매개변수로 받은 리뷰를 원핫 벡터로 변환시킴
  ## review (str) == 리뷰
  ## return = 변환된 원핫 벡터
  def vectorize(self, review):
    one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)

    for token in review.split(" "):

      ## string.punctuation == !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
      if token not in string.punctuation:
        one_hot[self.review_vocab.lookup_token(token)] = 1
    return one_hot
  
  @classmethod
  ## 데이터셋 데이터프레임에서 Vectorizer 객체를 만든다
  ## review_df (pandas.DataFrame) == 리뷰 데이터셋
  ## cutoff (int) == 빈도 기반 필터링 설정값
  ## return == ReviewVectorizer 객체
  def from_dataframe(cls, review_df, cutoff=25):
    review_vocab = Vocabulary(add_unk=True)
    rating_vocab = Vocabulary(add_unk=False)

    ## 점수 추가
    for rating in sorted(set(review_df.rating)):
      rating_vocab.add_token(rating)
    
    ## cutoff보다 많이 등장하는 단어 추가
    word_counts = Counter()
    for review in review_df.review:
      for word in review.split(" "):
        if word not in string.punctuation:
          word_counts[word] += 1
    
    for word, count in word_counts.items():
      if count > cutoff:
        review_vocab.add_token(word)
    
    return cls(review_vocab, rating_vocab)




    """ 직렬화된 딕셔너리에서 ReviewVectorizer 객체를 만듭니다
      
      매개변수:
          contents (dict): 직렬화된 딕셔너리
      반환값:
           ReviewVectorizer 클래스 객체
      """
  @classmethod
  def from_serializable(cls, contents):
      
    review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
    rating_vocab =  Vocabulary.from_serializable(contents['rating_vocab'])

    return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)

  def to_serializable(self):
    """ 캐싱을 위해 직렬화된 딕셔너리를 만듭니다
        
    반환값:
        contents (dict): 직렬화된 딕셔너리
    """
    return {'review_vocab': self.review_vocab.to_serializable(), 'rating_vocab': self.rating_vocab.to_serializable()}

In [24]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


**DataLoader**

미니 배치로 모아줌

Dataset, batch_size 등을 매개변수로 받아 순회

https://dojang.io/mod/page/view.php?id=2412

In [25]:
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"):

    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)


    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

### 3.6.4 퍼셉트론 분류기

ReviewClassifier 클래스는 파이토치의 Module 클래스를 상속하고 단일 출력을 만드는 Linear 층 하나 생성

- 이진 분류 문제
- 시그모이드 함수


In [26]:
cd /content/drive/MyDrive/nlp-with-pytorch/chapter_3

/content/drive/MyDrive/nlp-with-pytorch/chapter_3


In [27]:
class ReviewClassifier(nn.Module):

  ## num_features (int) == 입력 특성 
  def __init__(self, num_features):
    super(ReviewClassifier, self).__init__()
    self.fc1 = nn.Linear(in_features=num_features, out_features=1)

  
  ## forward 계산
  ## x_in (torch.Tensor) == 입력 데이터 텐서
  ## x_in.shape = (batch, num_features)
  ## apply_sigmoid (bool) == 시그모이드 활성화 함수
  def forward(self, x_in, apply_sigmoid=False):
    y_out = self.fc1(x_in).squeeze()
    if apply_sigmoid:
      y_out = torch.sigmoid(y_out)
    return y_out


## 3.6.5 모델 훈련

train_state == 훈련 과정 정보를 담음

In [28]:

def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [29]:

args = Namespace(
    # 날짜와 경로 정보
    frequency_cutoff=25,
    model_state_file='model.pth',
    review_csv='data/yelp/reviews_with_splits_lite.csv',
    # review_csv='data/yelp/reviews_with_splits_full.csv',
    save_dir='model_storage/ch3/yelp/',
    vectorizer_file='vectorizer.json',
    # 모델 하이퍼파라미터 없음



    # 훈련 하이퍼파라미터
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=100,
    seed=1337,



    # 실행 옵션
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("파일 경로: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
# CUDA 체크
if not torch.cuda.is_available():
    args.cuda = False

print("CUDA 사용여부: {}".format(args.cuda))

args.device = torch.device("cuda" if args.cuda else "cpu")

# 재현성을 위해 시드 설정
set_seed_everywhere(args.seed, args.cuda)

# 디렉토리 처리
handle_dirs(args.save_dir)

파일 경로: 
	model_storage/ch3/yelp/vectorizer.json
	model_storage/ch3/yelp/model.pth
CUDA 사용여부: True


**훈련 준비**

1. 훈련 상태 초기화

    복잡한 정보 다룰 수 있도록 args 객체 사용
    에포크 인덱스, 훈련 손실 리스트, 훈련 정확도, 검증 손실, 검증 정확도, 테스트 손실, 테스트 정확도 등

2. 데이터셋과 모델
  
  Vectorizer 생성

3. 손실 함수와 옵티마이저

  손실 함수 == `BCEWithLogitsLoss()` 사용 (BCE + sigmoid)

  옵티마이저 == "Adam"


In [30]:
## 데이터셋, 모델, 손실, 옵티마이저, 훈련 상태 딕셔너리 만들기


def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

## 훈련 state update

## early stopping
## Model checkpoint

## args (Namespace) == 매개변수
## model (nn.Module) == 훈련할 모델
## train_state (dic) == tratin state의 dictionary

def update_train_state(args, model, train_state):

  ## 초기 상태 == 초기 모델 저장하고 early stopping 값 False로 설정해줌
  if train_state['epoch_index'] == 0:
    torch.save(model.state_dict(), train_state['model_filename'])
    train_state['stop_early'] = False

  ## 훈련중 ...
  elif train_state['epoch_index'] >= 1:
    loss_tm1, loss_t = train_state['val_loss'][-2:]

    ## 손실이 나빠지면 early stopping ++
    if loss_t >= train_state['early_stopping_best_val']:
      train_state['early_stopping_step'] += 1
    ## 손실 좋아지면 모델 저장하고 early stopping = 0
    else:
      if loss_t < train_state['early_stopping_best_val']:
        torch.save(model.state_dict(), train_state['model_filename'])
      
      train_state['early_stopping_step'] = 0
    

    ## early stopping 할거야?
    train_state['stop_early'] = train_state['early_stopping_step'] >= args.early_stopping_criteria
  
  return train_state

def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [31]:

# 체크포인트에서 훈련을 다시 시작
if args.reload_from_files:
    print("데이터셋과 Vectorizer를 로드합니다")
    dataset = ReviewDataset.load_dataset_and_load_vectorizer(args.review_csv,
                                                            args.vectorizer_file)
# 처음부터 만들기
else:
    print("데이터셋을 로드하고 Vectorizer를 만듭니다")
    dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_csv)
    dataset.save_vectorizer(args.vectorizer_file)   
vectorizer = dataset.get_vectorizer()

classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab))
classifier = classifier.to(args.device)

loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)
train_state = make_train_state(args)

데이터셋을 로드하고 Vectorizer를 만듭니다


**훈련 반복**

In [32]:
## 진행바를 위한 함수함수함수들~~~
epoch_bar = tqdm.notebook.tqdm(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)


## 에포크 횟수만큼 반복!
for epoch_index in range(args.num_epochs):
  train_state['epoch_index'] = epoch_index

  
  ## ------------------------------------------------------------------

  ## train data의 배치 생성
  dataset.set_split('train')
  batch_generator = generate_batches(dataset=dataset,
                                     batch_size = args.batch_size,
                                     device=args.device)

  ## loss와 accuracy 초기화
  running_loss = 0.0
  running_acc = 0.0
  ## 모델이 train 중에 있다 == 파라미터를 바꿀 수 있다
  classifier.train()

  ## train data 돌기
  for batch_index, batch_dict in enumerate(batch_generator):


    ## 1. Gradient 0으로 초기화
    optimizer.zero_grad()

    ## 2. 출력 계산
    y_pred = classifier(x_in=batch_dict['x_data'].float())

    ## 3. 손실 계산
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()

    ## ???????????
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    ## 4. 손실 기반으로 역전파
    loss.backward()

    ## 5. 옵티마이저로 업데이트
    optimizer.step()

    

    ## 정확도 계산
    acc_t = compute_accuracy(y_pred=y_pred, y_target=batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

    # 진행 바 업데이트
    train_bar.set_postfix(loss=running_loss,
                          acc=running_acc, 
                          epoch=epoch_index)
    train_bar.update()
  

  train_state['train_loss'].append(running_loss)
  train_state['train_acc'].append(running_acc)



  ## ------------------------------------------------------------------




  dataset.set_split('val')
  batch_generator = generate_batches(dataset,
                                     batch_size = args.batch_size,
                                     device = args.device)
  running_loss = 0.
  running_acc = 0.
  ## train()과 반대로 파라미터 업데이트 못하게 하고 드롭아웃도 없앰
  ## 손실 계산 안하고 그래디언트 전파 안함?? 아하아하
  classifier.eval()

  for batch_index, batch_dict in enumerate(batch_generator):

    ## 1. 출력 계산
    y_pred = classifier(x_in=batch_dict['x_data'].float())
    ## 2. 손실 계산
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)
    ## 3. 정확도 계산
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

    ## 진행바 업데이트
    val_bar.set_postfix(loss=running_loss,
                        acc=running_acc,
                        epoch=epoch_index)
    val_bar.update()

  train_state['val_loss'].append(running_loss)
  train_state['val_acc'].append(running_acc)

  train_state = update_train_state(args=args, model=classifier,
                                   train_state=train_state)
  
  scheduler.step(train_state['val_loss'][-1])

  train_bar.n = 0
  val_bar.n = 0
  epoch_bar.update()

  if train_state['stop_early']:
    break

  train_bar.n = 0
  val_bar.n = 0
  epoch_bar.update()



training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/3062 [00:00<?, ?it/s]

split=val:   0%|          | 0/656 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

### 3.6.6 평가, 추론, 분석

1. 테스트 데이터로 평가하기


In [33]:
### 1. 테스트 데이터로 평가하기

classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 출력을 계산합니다
    y_pred = classifier(x_in=batch_dict['x_data'].float())

    # 손실을 계산합니다
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도를 계산합니다
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc


print("테스트 손실: {:.3f}".format(train_state['test_loss']))
print("테스트 정확도: {:.2f}".format(train_state['test_acc']))


테스트 손실: 0.186
테스트 정확도: 93.19


2. 새로운 데이터로 추론해보고 평가하기


In [34]:
## 2. 새로운 데이터로 추론해보고 평가하기

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text


def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
    review = preprocess_text(review)
    
    vectorized_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorized_review.view(1, -1))
    
    probability_value = torch.sigmoid(result).item()
    index = 1
    if probability_value < decision_threshold:
        index = 0

    return vectorizer.rating_vocab.lookup_index(index)

In [35]:

test_review = "this is a pretty awesome book"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

this is a pretty awesome book -> positive


3. 모델 가중치 분석

In [36]:
# 가중치 정렬
fc1_weights = classifier.fc1.weight.detach()[0]
_, indices = torch.sort(fc1_weights, dim=0, descending=True)
indices = indices.numpy().tolist()

# 긍정적인 상위 20개 단어
print("긍정 리뷰에 영향을 미치는 단어:")
print("--------------------------------------")
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))
    
print("====\n\n\n")

# 부정적인 상위 20개 단어
print("부정 리뷰에 영향을 미치는 단어:")
print("--------------------------------------")
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

긍정 리뷰에 영향을 미치는 단어:
--------------------------------------
exceeded
hesitate
delicious
pleasantly
excellent
nexcellent
perfection
downside
yum
amazing
disappoint
deliciousness
yummm
incredible
fantastic
nhighly
hooked
awesome
drawback
ndelicious
====



부정 리뷰에 영향을 미치는 단어:
--------------------------------------
poisoning
worst
mediocre
meh
underwhelmed
slowest
overrated
bland
tasteless
downhill
flavorless
horrible
terrible
disappointing
inedible
unacceptable
unprofessional
unimpressed
ception
awful
